In [1]:
import sys
paths_to_add = ["/homes/vk223/ProjectFlood", "/home/vkhandekar/project_flood"]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)
print(sys.path)

from model_runs.model_run_helpers import *
from model_runs.distributed_gpu_helpers import *
from visualisations.visualisation_helpers import *
import os
print(os.getcwd())


['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages', '/homes/vk223/ProjectFlood', '/home/vkhandekar/project_flood']
/homes/vk223/ProjectFlood/evaluation


In [2]:
from evaluation.model_evaluation_helpers import *
os.environ["PROJECT_FLOOD_DATA"] = "/homes/vk223/ProjectFlood/static/imperial_data_paths.json"
os.environ["PROJECT_FLOOD_CORE_PATHS"] = "/homes/vk223/ProjectFlood/static/imperial_core_paths.json"
print(os.environ["PROJECT_FLOOD_DATA"])
resolution = 256


/homes/vk223/ProjectFlood/static/imperial_data_paths.json


In [3]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
torch.manual_seed(42)
# Load newly trained best, final models
final_sep_branch_model, _, _, final_sep_branch_params = load_checkpoint("/homes/vk223/ProjectFlood/models/saved_models/ConvLSTMMerged_epochs500_batchsize16_lr0p001_precedingrainfall1_dropout0p3_outputchannels8_convblocklayers2_convLSTMlayers2_optimRMSprop_criterionBCELoss_transformsFalse_res256_20240824_500.pt")
final_merged_model, _, _, final_merged_params = load_checkpoint("/homes/vk223/ProjectFlood/models/saved_models/ConvLSTMSeparateBranches_epochs500_batchsize32_lr0p001_precedingrainfall1_dropout0p5_outputchannels8_convblocklayers2_convLSTMlayers1_optimRMSprop_criterionBCELoss_transformsFalse_res256_20240824_500.pt")

dict_keys(['epochs', 'batchsize', 'lr', 'precedingrainfall', 'dropout', 'outputchannels', 'convblocklayers', 'convLSTMlayers', 'optim', 'criterion', 'transforms', 'res'])
dict_keys(['epochs', 'batchsize', 'lr', 'precedingrainfall', 'dropout', 'outputchannels', 'convblocklayers', 'convLSTMlayers', 'optim', 'criterion', 'transforms', 'res'])


/homes/vk223/ProjectFlood/evaluation/model_evaluation_helpers.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath)
/vol/bitbucket/vk223/pro

In [6]:
sep_branch_test_dataloader = get_dataloader("test_labels_path", resolution=256, preceding_rainfall_days=final_sep_branch_params['precedingrainfall'], forecast_rainfall_days=1, 
                                                transform=None, batch_size=16, shuffle=False, num_workers=4)
    
merged_test_dataloader = get_dataloader("test_labels_path", resolution=256, preceding_rainfall_days=final_merged_params['precedingrainfall'], forecast_rainfall_days=1, 
                                            transform=None, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
from visualisations.visualisation_helpers import *
# Plot image
with torch.no_grad():
    size = 3

    sep_branch_flooded_images = []
    sep_branch_non_flooded_images = []
    for inputs, targets, flooded in sep_branch_test_dataloader:
        sep_branch_outputs = final_sep_branch_model(inputs)
        for i in range(len(flooded)):
            if flooded[i] == 1 and len(sep_branch_flooded_images) < size:
                sep_branch_flooded_images.append((sep_branch_outputs[i], targets[i], flooded[i]))
            elif flooded[i] == 0 and len(sep_branch_non_flooded_images) < size:
                sep_branch_non_flooded_images.append((sep_branch_outputs[i], targets[i], flooded[i]))
            # Stop if we've collected 4 images in each category
            if len(sep_branch_flooded_images) >= size and len(sep_branch_non_flooded_images) >= size:
                break
        if len(sep_branch_flooded_images) >= 4 and len(sep_branch_non_flooded_images) >= 4:
            break

    merged_flooded_images = []
    merged_non_flooded_images = []
    for inputs, targets, flooded in merged_test_dataloader:
        merged_outputs = final_merged_model(inputs)
        for i in range(len(flooded)):
            if flooded[i] == 1 and len(merged_flooded_images) < size:
                merged_flooded_images.append((merged_outputs[i], targets[i], flooded[i]))
            elif flooded[i] == 0 and len(merged_non_flooded_images) < size:
                merged_non_flooded_images.append((merged_outputs[i], targets[i], flooded[i]))
            # Stop if we've collected 4 images in each category
            if len(merged_flooded_images) >= size and len(merged_non_flooded_images) >= size:
                break
        if len(merged_flooded_images) >= 4 and len(merged_non_flooded_images) >= 4:
            break
    
    with open(os.environ["PROJECT_FLOOD_CORE_PATHS"]) as core_config_file:
        core_config = json.load(core_config_file)
    dimension_string = core_config[f"rainfall_reprojection_master_256"]
    match = re.search(r'_(\d+)_(\d+)\.tif$', dimension_string)
    new_dimension_right, new_dimension_bottom = int(match.group(1)), int(match.group(2))


    selected_sep_branch_outputs = [img[0] for img in sep_branch_flooded_images + sep_branch_non_flooded_images]
    selected_sep_branch_outputs = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_sep_branch_outputs] #crop


    selected_merged_outputs = [img[0] for img in merged_flooded_images + merged_non_flooded_images]
    selected_merged_outputs = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_merged_outputs] #crop

    model_names = ['Branched', 'Merged']
    selected_model_outputs = [selected_sep_branch_outputs, selected_merged_outputs]

    selected_targets = [img[1] for img in sep_branch_flooded_images + sep_branch_non_flooded_images]
    selected_targets = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_targets] #crop


    selected_targets_flooded = [img[2] for img in sep_branch_flooded_images + sep_branch_non_flooded_images] #boolean for flooded or not

    # Do plotting
    with open(os.environ["PROJECT_FLOOD_DATA"]) as data_config_file:
        data_config = json.load(data_config_file)
    plot_filename = os.path.join(data_config["model_results_path"], "final_plots.png")
    plot_final_model_output_vs_label(model_names, selected_model_outputs, selected_targets, selected_targets_flooded, plot_filename)

In [7]:
with open(os.environ["PROJECT_FLOOD_DATA"]) as data_config_file:
    data_config = json.load(data_config_file)
with open(os.environ["PROJECT_FLOOD_CORE_PATHS"]) as core_config_file:
    core_config = json.load(core_config_file)
dimension_string = core_config[f"rainfall_reprojection_master_{resolution}"]
match = re.search(r'_(\d+)_(\d+)\.tif$', dimension_string)
new_dimension_right, new_dimension_bottom = int(match.group(1)), int(match.group(2))


# model = final_sep_branch_model.to(device)
model = final_sep_branch_model
model.eval()
criterion = getattr(torch.nn, final_sep_branch_params['criterion'])()

# Load the True/False mask
mask = np.load(os.path.join(data_config["model_results_path"], "perm_water_mask.npy"))
print(mask.shape)

(256, 203)


In [8]:
thresholds = np.arange(0, 1.05, 0.05)
confusion_matrices = {thr: np.zeros((2, 2)) for thr in thresholds}
precision_scores = {thr: 0 for thr in thresholds}
recall_scores = {thr: 0 for thr in thresholds}
accuracy_scores = {thr: 0 for thr in thresholds}

total_samples = 0

total_loss = 0
total_rmse = 0
total_mae = 0
total_psnr = 0
total_ssim = 0

with torch.no_grad():
    for inputs, labels, _ in sep_branch_test_dataloader: #batch size is first dim. (BXY)
        # inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)
        inputs, labels = inputs.to(torch.float32), labels.to(torch.float32)
        outputs = model(inputs)

        # Crop the outputs and labels as required
        cropped_outputs = outputs[..., 0:new_dimension_bottom, 0:new_dimension_right]  
        cropped_labels = labels[..., 0:new_dimension_bottom, 0:new_dimension_right]
        # print(cropped_outputs.shape)

        # Apply mask (retain False values)
        mask_torch = torch.from_numpy(mask)
        mask_torch = mask_torch.unsqueeze(0)
        mask_torch = mask_torch.expand(cropped_outputs.size(0), -1, -1)

        masked_outputs = cropped_outputs[~mask_torch]
        masked_labels = cropped_labels[~mask_torch]

        # 1. Calculate PSNR and SSIM on cropped (but not masked) images
        psnr_value = PSNR(cropped_outputs, cropped_labels)
        ssim_value = SSIM(cropped_outputs, cropped_labels)
        total_psnr += psnr_value.item()
        total_ssim += ssim_value.item()

        # 2. Calculate RMSE and MAE on masked images
        rmse_loss = RMSELoss(masked_outputs, masked_labels)
        mae_loss = MAELoss(masked_outputs, masked_labels)

        total_rmse += rmse_loss.item()
        total_mae += mae_loss.item()

        # Calculate the total loss
        loss = criterion(masked_outputs, masked_labels).item()
        total_loss += loss
        total_samples += masked_labels.size(0)

        # Apply different thresholds and calculate confusion matrices
        for thr in thresholds:
            binary_output = (masked_outputs > thr).float()
            binary_output = binary_output.cpu().numpy().flatten()
            binary_labels = masked_labels.cpu().numpy().flatten()

            cm = confusion_matrix(binary_labels, binary_output, labels=[0, 1])
            confusion_matrices[thr] += cm
            precision_scores[thr] += precision_score(binary_labels, binary_output, zero_division=0)
            recall_scores[thr] += recall_score(binary_labels, binary_output, zero_division=0)

            # Calculate accuracy at this threshold
            correct_predictions = (binary_output == binary_labels).sum()
            accuracy_scores[thr] += correct_predictions

torch.Size([16, 256, 203])


/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


torch.Size([16, 256, 203])


/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


torch.Size([16, 256, 203])


/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


torch.Size([13, 256, 203])


/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


In [9]:
# Calculate averages
avg_rmse = total_rmse / len(sep_branch_test_dataloader)
avg_mae = total_mae / len(sep_branch_test_dataloader)
avg_psnr = total_psnr / len(sep_branch_test_dataloader)
avg_ssim = total_ssim / len(sep_branch_test_dataloader)
average_loss = total_loss / total_samples

# Calculate the final precision and recall
for thr in thresholds:
    precision_scores[thr] /= total_samples
    recall_scores[thr] /= total_samples
    accuracy_scores[thr] /= total_samples

metric_accumulator = {
    'average_rmse': avg_rmse,
    'average_mae': avg_mae,
    'average_psnr': avg_psnr,
    'average_ssim': avg_ssim,
    'average_loss': average_loss,
    'confusion_matrices': confusion_matrices, #tn, fp, fn, tp
    'precision_scores': precision_scores,
    'recall_scores': recall_scores,
    'accuracy_scores': accuracy_scores
}

In [10]:
metric_accumulator

{'average_rmse': 0.19500676915049553,
 'average_mae': 0.27970991283655167,
 'average_psnr': 14.693440914154053,
 'average_ssim': 0.6088900566101074,
 'average_loss': 2.1045159267313747e-07,
 'confusion_matrices': {np.float64(0.0): array([[      0., 2344219.],
         [      0.,   92365.]]),
  np.float64(0.05): array([[1824063.,  520156.],
         [   6616.,   85749.]]),
  np.float64(0.1): array([[1999085.,  345134.],
         [  12620.,   79745.]]),
  np.float64(0.15000000000000002): array([[2081756.,  262463.],
         [  18274.,   74091.]]),
  np.float64(0.2): array([[2132962.,  211257.],
         [  23113.,   69252.]]),
  np.float64(0.25): array([[2169812.,  174407.],
         [  27390.,   64975.]]),
  np.float64(0.30000000000000004): array([[2196761.,  147458.],
         [  31219.,   61146.]]),
  np.float64(0.35000000000000003): array([[2217885.,  126334.],
         [  34811.,   57554.]]),
  np.float64(0.4): array([[2234717.,  109502.],
         [  38226.,   54139.]]),
  np.floa